Import Modules

In [14]:
import pandas as pd
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading the Dataset

In [15]:
df = pd.read_csv('/content/encoded-spam.csv')

In [16]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [17]:
df = df[['v2' , 'v1']]
df = df.rename(columns={'v2': 'messages' , 'v1': 'label'})
df.head()

,messages,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


Processing The Dataset

In [18]:
# checking for null values
df.isnull().sum()

messages    0
label       0
dtype: int64

In [19]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
  text = text.lower()
  text = re.sub(r'^0-9a-zA-Z' , ' ', text)
  text = re.sub(r'\s+', ' ', text)
  text = " ".join(word for word in text.split() if word not in STOPWORDS)
  return text

In [20]:
# clean the messages
df['clean_text'] = df['messages'].apply(clean_text)
df.head()

,messages,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,"go jurong point, crazy.. available bugis n gre..."
1,Ok lar... Joking wif u oni...,ham,ok lar... joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,ham,u dun say early hor... u c already say...
4,"Nah I don't think he goes to usf, he lives aro...",ham,"nah think goes usf, lives around though"


Input Split

In [21]:
X = df['clean_text']
Y = df['label']

Model Training

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def classify(model, X, Y):
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42, shuffle=True, stratify=Y)
  pipeline_model = Pipeline([('vect', CountVectorizer()),
                             ('tfidf', TfidfTransformer()),
                             ('clf', model)])
  pipeline_model.fit(X_train, Y_train)

  print('Accuracy:', pipeline_model.score(X_test, Y_test)*100)

 #  cv_score = cross_val_score(model, X, Y, cv=5)
 #  print("CV Score:", np.mean(cv_score)*100)
  Y_pred = pipeline_model.predict(X_test)
  print(classification_report(Y_test, Y_pred))


In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model, X, Y)

Accuracy: 96.8413496051687
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1206
        spam       1.00      0.76      0.87       187

    accuracy                           0.97      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.97      0.97      0.97      1393



In [24]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classify(model, X, Y)

Accuracy: 96.55419956927494
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1206
        spam       1.00      0.74      0.85       187

    accuracy                           0.97      1393
   macro avg       0.98      0.87      0.92      1393
weighted avg       0.97      0.97      0.96      1393



In [25]:
from sklearn.svm import SVC
model = SVC(C=10)
classify(model, X, Y)

Accuracy: 98.27709978463747
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1206
        spam       1.00      0.87      0.93       187

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



In [26]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model, X, Y)

Accuracy: 97.34386216798278
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1206
        spam       1.00      0.80      0.89       187

    accuracy                           0.97      1393
   macro avg       0.99      0.90      0.94      1393
weighted avg       0.97      0.97      0.97      1393

